This is the messy workbook where I develop my AI models. If you want to try out the training process for yourself, look at the replication book.


Dataset:

We will be using the Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation dataset because it has extrenal eye picutres, allowing users to take pictures of thier potienal eye disease without needed speical equipment. Link to the dataset can be found here: https://data.mendeley.com/datasets/n9zp473wfw/2

In [1]:
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/n9zp473wfw-2.zip

--2025-08-24 18:00:02--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/n9zp473wfw-2.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 3.5.64.49, 52.92.33.162, 52.218.91.80, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|3.5.64.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42753308 (41M) [application/zip]
Saving to: ‘n9zp473wfw-2.zip’

n9zp473wfw-2.zip    100%[===================>]  40.77M  19.5MB/s    in 2.1s    

2025-08-24 18:00:04 (19.5 MB/s) - ‘n9zp473wfw-2.zip’ saved [42753308/42753308]



In [2]:
!unzip n9zp473wfw-2.zip

Archive:  n9zp473wfw-2.zip
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Duplicate.ipynb  
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Eye_Diseases_Classifications_Pre_Processing_Dataset.ipynb  
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Cataract/10.jpg  
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Cataract/100.jpg  
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Cataract/1.jpg  
  inflating: Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/Cataract/101.jpg  


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
import numpy as np
import os
from tensorflow.keras.applications import EfficientNetV2M
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.applications import EfficientNetV2XL


ImportError: cannot import name 'EfficientNetV2XL' from 'tensorflow.keras.applications' (/usr/local/lib/python3.12/dist-packages/keras/_tf_keras/keras/applications/__init__.py)

In [ ]:
DATA_DIR = "/content/Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_FROZEN = 5
EPOCHS_FINE = 20
NUM_CLASSES = 5
TRAIN_DIR = "dataset_train"
VAL_DIR = "dataset_val"
TEST_SIZE = 0.2
RANDOM_STATE = 42

In [ ]:
!rm -rf "/content/Image Dataset on Eye Diseases Classification (Uveitis, Conjunctivitis, Cataract, Eyelid) with Symptoms and SMOTE Validation/.ipynb_checkpoints"
!rm -rf /content/dataset_train/
!rm -rf /content/dataset_val/

In [ ]:
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

classes = os.listdir(DATA_DIR)
for cls in classes:
    os.makedirs(os.path.join(TRAIN_DIR, cls), exist_ok=True)
    os.makedirs(os.path.join(VAL_DIR, cls), exist_ok=True)

for cls in classes:
    cls_path = os.path.join(DATA_DIR, cls)
    images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    train_imgs, val_imgs = train_test_split(images, test_size=TEST_SIZE, random_state=RANDOM_STATE)

    # Move images
    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(TRAIN_DIR, cls, img))

    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(VAL_DIR, cls, img))

print("Train/test split completed!")

Train/test split completed!


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.6,1.4],
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

val_generator = train_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

Found 1837 images belonging to 5 classes.
Found 461 images belonging to 5 classes.


In [ ]:
labels = train_generator.classes
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(labels),
                                                 y=labels)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

Class weights: {0: np.float64(0.8445977011494253), 1: np.float64(1.2891228070175438), 2: np.float64(0.8747619047619047), 3: np.float64(0.7078998073217726), 4: np.float64(2.0640449438202246)}


In [ ]:
base_model = EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze base initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
history_frozen = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FROZEN,
    class_weight=class_weights
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.1966 - loss: 1.6082 - val_accuracy: 0.1475 - val_loss: 1.6133
Epoch 2/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 608ms/step - accuracy: 0.2012 - loss: 1.6094 - val_accuracy: 0.1952 - val_loss: 1.6118
Epoch 3/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 613ms/step - accuracy: 0.1913 - loss: 1.6243 - val_accuracy: 0.1887 - val_loss: 1.6092
Epoch 4/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 590ms/step - accuracy: 0.1932 - loss: 1.5945 - val_accuracy: 0.1822 - val_loss: 1.6091
Epoch 5/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 612ms/step - accuracy: 0.1994 - loss: 1.6168 - val_accuracy: 0.1475 - val_loss: 1.6105


In [ ]:
for layer in base_model.layers[-125:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),  # smaller LR for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1
)

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE,
    class_weight=class_weights,
    callbacks=[checkpoint]
)

Epoch 1/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1863 - loss: 1.6234
Epoch 1: val_accuracy improved from -inf to 0.15835, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.1869 - loss: 1.6233 - val_accuracy: 0.1584 - val_loss: 1.6136
Epoch 2/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.2501 - loss: 1.5683
Epoch 2: val_accuracy improved from 0.15835 to 0.23861, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 680ms/step - accuracy: 0.2500 - loss: 1.5687 - val_accuracy: 0.2386 - val_loss: 1.5936
Epoch 3/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - accuracy: 0.2658 - loss: 1.5847
Epoch 3: val_accuracy improved from 0.23861 to 0.26247, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 676ms/step - accuracy: 0.2660 - loss: 1.5845 - val_accuracy: 0.2625 - val_loss: 1.5508
Epoch 4/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.3166 - loss: 1.5686
Epoch 4: val_accuracy improved from 0.26247 to 0.27766, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 675ms/step - accuracy: 0.3164 - loss: 1.5683 - val_accuracy: 0.2777 - val_loss: 1.5328
Epoch 5/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.3193 - loss: 1.5223
Epoch 5: val_accuracy improved from 0.27766 to 0.31670, saving model to best_model.h5


...

58/58 ━━━━━━━━━━━━━━━━━━━━ 46s 754ms/step - accuracy: 0.3737 - loss: 1.4751 - val_accuracy: 0.3818 - val_loss: 1.4364
Epoch 10/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 503ms/step - accuracy: 0.3797 - loss: 1.4786
Epoch 10: val_accuracy did not improve from 0.38178
58/58 ━━━━━━━━━━━━━━━━━━━━ 75s 636ms/step - accuracy: 0.3798 - loss: 1.4783 - val_accuracy: 0.3601 - val_loss: 1.4272
Epoch 11/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.3767 - loss: 1.4835
Epoch 11: val_accuracy did not improve from 0.38178
58/58 ━━━━━━━━━━━━━━━━━━━━ 37s 638ms/step - accuracy: 0.3767 - loss: 1.4837 - val_accuracy: 0.3818 - val_loss: 1.4172
Epoch 12/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.3526 - loss: 1.4624
Epoch 12: val_accuracy improved from 0.38178 to 0.38829, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 722ms/step - accuracy: 0.3531 - loss: 1.4625 - val_accuracy: 0.3883 - val_loss: 1.3764
Epoch 13/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.3591 - loss: 1.4734
Epoch 13: val_accuracy did not improve from 0.38829
58/58 ━━━━━━━━━━━━━━━━━━━━ 37s 637ms/step - accuracy: 0.3595 - loss: 1.4732 - val_accuracy: 0.3623 - val_loss: 1.4556
Epoch 14/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 509ms/step - accuracy: 0.3849 - loss: 1.4608
Epoch 14: val_accuracy did not improve from 0.38829
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 621ms/step - accuracy: 0.3848 - loss: 1.4607 - val_accuracy: 0.3579 - val_loss: 1.4104
Epoch 15/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - accuracy: 0.3762 - loss: 1.4727
Epoch 15: val_accuracy improved from 0.38829 to 0.40564, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 676ms/step - accuracy: 0.3762 - loss: 1.4725 - val_accuracy: 0.4056 - val_loss: 1.3870
Epoch 16/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 526ms/step - accuracy: 0.3752 - loss: 1.4582
Epoch 16: val_accuracy improved from 0.40564 to 0.41649, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 692ms/step - accuracy: 0.3750 - loss: 1.4581 - val_accuracy: 0.4165 - val_loss: 1.4101
Epoch 17/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.3862 - loss: 1.4497
Epoch 17: val_accuracy did not improve from 0.41649
58/58 ━━━━━━━━━━━━━━━━━━━━ 37s 641ms/step - accuracy: 0.3863 - loss: 1.4496 - val_accuracy: 0.3666 - val_loss: 1.4483
Epoch 18/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - accuracy: 0.3929 - loss: 1.4277
Epoch 18: val_accuracy did not improve from 0.41649
58/58 ━━━━━━━━━━━━━━━━━━━━ 37s 647ms/step - accuracy: 0.3929 - loss: 1.4277 - val_accuracy: 0.3210 - val_loss: 1.5636
Epoch 19/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.3929 - loss: 1.4365
Epoch 19: val_accuracy improved from 0.41649 to 0.42082, saving model to best_model.h5


58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 725ms/step - accuracy: 0.3929 - loss: 1.4364 - val_accuracy: 0.4208 - val_loss: 1.3509
Epoch 20/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.3999 - loss: 1.4351
Epoch 20: val_accuracy did not improve from 0.42082
58/58 ━━━━━━━━━━━━━━━━━━━━ 38s 647ms/step - accuracy: 0.3999 - loss: 1.4348 - val_accuracy: 0.3579 - val_loss: 1.4709


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# -------------------
# Parameters
# -------------------
IMG_SIZE = (384, 384)   # EfficientNetV2-S input size
BATCH_SIZE = 32
EPOCHS = 30
TRAIN_DIR = "dataset_train"
VAL_DIR = "dataset_val"

# -------------------
# Data Augmentation
# -------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

# -------------------
# Class Weights
# -------------------
labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)
class_weights = dict(enumerate(class_weights))

# -------------------
# Model
# -------------------
base_model = EfficientNetV2S(weights="imagenet", include_top=False, input_shape=IMG_SIZE+(3,))
base_model.trainable = False  # freeze initially

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(train_generator.num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

# -------------------
# Compile
# -------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# -------------------
# Callbacks
# -------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
]

# -------------------
# Step 1: Train frozen
# -------------------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    class_weight=class_weights,
    callbacks=callbacks
)

# -------------------
# Step 2: Fine-tune (unfreeze last layers)
# -------------------
base_model.trainable = True
for layer in base_model.layers[:-100]:  # freeze all but last 100 layers
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)


Found 1837 images belonging to 5 classes.
Found 461 images belonging to 5 classes.
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2283 - loss: 1.6030
Epoch 1: val_accuracy improved from -inf to 0.35141, saving model to best_model.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.2291 - loss: 1.6025 - val_accuracy: 0.3514 - val_loss: 1.5298 - learning_rate: 0.0010
Epoch 2/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3276 - loss: 1.5476
Epoch 2: val_accuracy improved from 0.35141 to 0.35792, saving model to best_model.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.3277 - loss: 1.5474 - val_accuracy: 0.3579 - val_loss: 1.4965 - learning_rate: 0.0010
Epoch 3/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3699 - loss: 1.4770
Epoch 3: val_accuracy improved from 0.35792 to 0.37744, saving model to best_model.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.3698 - loss: 1.4774 - val_accuracy: 0.3774 - val_loss: 1.4658 - learning_rate: 0.0010
Epoch 4/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.35